#### Imports

In [1]:
import os
import gensim
import nltk
import numpy as np
import pickle
from keras.models import Sequential, load_model
from keras.layers.recurrent import LSTM,SimpleRNN,GRU
from keras.layers import TimeDistributed, Activation, Dense
from keras.utils import np_utils
from scipy import spatial
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


#### Importing pickled questions and answers

In [2]:
with open('Notebook/conversation.pickle', 'rb') as f:
    vec_x,vec_y=pickle.load(f)    
    
vec_x=np.array(vec_x,dtype=np.float64)
vec_y=np.array(vec_y,dtype=np.float64)

In [4]:
vec_y.shape

(6226, 15, 300)

#### Split data and define model layers

In [32]:
x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.3, random_state=42)

In [35]:
n_in_out = 4358
n_hidden = 100
n_samples = 2297
n_timesteps = 400

model2 = Sequential()
# `return_sequences` controls whether to copy the input automatically
model2.add(GRU( n_hidden, input_dim = n_in_out, return_sequences=True))
model2.add(Dense(n_in_out, input_dim = n_hidden))
model2.compile(loss='mse', optimizer='rmsprop')

/home/kenny/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/kenny/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(100, return_sequences=True, input_shape=(None, 435...)`


In [4]:
model=Sequential()
model.add(LSTM(output_dim=300, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

/home/kenny/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  from ipykernel import kernelapp as app
/home/kenny/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  app.launch_new_instance()
/home/kenny/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
/home/kenny/anaconda3/lib/python3.6/site-packages/ip

#### Train model on data

In [9]:
model.fit(x_train, y_train, nb_epoch=5,validation_data=(x_test, y_test))
model.save('LSTM500.h5')

/home/kenny/anaconda3/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 4358 samples, validate on 1868 samples
Epoch 1/5
4358/4358 [==============================] - 43s - loss: -9.8088e-04 - acc: 0.0329 - val_loss: -9.8083e-04 - val_acc: 0.0278

#### Load Google word2vec as vectors for decoding model outputs and making predictions

In [6]:
mod = gensim.models.KeyedVectors.load_word2vec_format('/home/kenny/Notebook/GoogleNews-vectors-negative300.bin', binary=True)

In [7]:
predictions=model.predict(x_test)

In [8]:
[mod.most_similar([predictions[10][i]])[0] for i in range(15)]

[('RUSH_Okay', 0.505364179611206),
 ('ought', 0.496914803981781),
 ('ought', 0.49452176690101624),
 ('ought', 0.4933737814426422),
 ('ought', 0.4925598204135895),
 ('ought', 0.4917911887168884),
 ('ought', 0.4906594753265381),
 ('ought', 0.48921531438827515),
 ('ought', 0.48705795407295227),
 ('ought', 0.4835759997367859),
 ('ought', 0.4804413318634033),
 ('ought', 0.47531694173812866),
 ('ought', 0.46747350692749023),
 ('ought', 0.45696133375167847),
 ('ought', 0.4451048970222473)]

In [15]:
model=load_model('LSTM500.h5')

while(True):
    x=input("Enter the message:");
    sentend=np.ones((300,),dtype=np.float32) 

    sent=nltk.word_tokenize(x.lower())
    sentvec = [mod[w] for w in sent if w in mod.vocab]

    sentvec[14:]=[]
    sentvec.append(sentend)
    if len(sentvec)<15:
        for i in range(15-len(sentvec)):
            sentvec.append(sentend) 
    sentvec=np.array([sentvec])
    
    predictions = model.predict(sentvec)
    outputlist=[mod.most_similar([predictions[0][i]])[0][0] for i in range(15)]
    output=' '.join(outputlist)
    print(output)

Enter the message:hi
anyway ought ought ought ought ought ought ought ought ought this this this this going
Enter the message:how old are you
anyway ought ought ought ought ought ought ought ought this this this this this going
Enter the message:what time is it
anyway ought ought ought ought ought ought ought ought this this this this this going
Enter the message:how to fix a light bulb
anyway ought ought ought ought ought ought ought ought ought this this this this going
Enter the message:it doesn't really matter
anyway ought ought ought ought ought ought ought ought ought this this this this going


KeyboardInterrupt: 